## 20. JSONデータの読み込み

In [1]:
import urllib

url  = "http://www.cl.ecei.tohoku.ac.jp/nlp100/data/jawiki-country.json.gz"
urllib.request.urlretrieve(url, 'jawiki-country.json.gz')

('jawiki-country.json.gz', <http.client.HTTPMessage at 0x10e12cf60>)

In [2]:
import gzip,json
eng = {}
with gzip.open('jawiki-country.json.gz','rt') as f:
    for line in f:
        j = json.loads(line)
        if j['title']=='イギリス':
            eng = j

#出力省略

## 21. カテゴリ名を含む行を抽出

In [3]:
import re
for line in eng['text'].split('\n'):
    match = re.search('Category',line)
    if match:
        print(line)

[[Category:イギリス|*]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国]]
[[Category:海洋国家]]
[[Category:君主国]]
[[Category:島国|くれいとふりてん]]
[[Category:1801年に設立された州・地域]]


## 22. カテゴリ名の抽出

In [4]:
for line in eng['text'].split('\n'):
    match = re.search('Category:(.*?)[\|\]]',line)
    if match:
        print(match.group(1))

イギリス
英連邦王国
G8加盟国
欧州連合加盟国
海洋国家
君主国
島国
1801年に設立された州・地域


## 23. セクション構造

In [5]:
sec = [0,0,0]
for line in eng['text'].split('\n'):
    match = re.search('==(.*)==',line)
    if match:
        matched = match.group(0)
        match2 = re.search('[^=]+',matched)
        level = match2.start(0)-2
        for i in range(level+1,3):
            sec[i]=0
        sec[level]+=1
        print(".".join(map(str,sec[:level+1])),end=' ')
        print(match2.group(0))

1 国名
2 歴史
3 地理
3.1 気候
4 政治
5 外交と軍事
6 地方行政区分
6.1 主要都市
7 科学技術
8 経済
8.1 鉱業
8.2 農業
8.3 貿易
8.4 通貨
8.5 企業
9 交通
9.1 道路
9.2 鉄道
9.3 海運
9.4 航空
10 通信
11 国民
11.1 言語
11.2 宗教
11.3  婚姻 
11.4 教育
12 文化
12.1 食文化
12.2 文学
12.3  哲学 
12.4 音楽
12.4.1 イギリスのポピュラー音楽
12.5 映画
12.6 コメディ
12.7 国花
12.8 世界遺産
12.9 祝祭日
13 スポーツ
13.1 サッカー
13.2 競馬
13.3 モータースポーツ
14 脚注
15 関連項目
16 外部リンク


## 24. ファイル参照の抽出

In [6]:
for m in re.finditer('File:(.*?)\|',eng['text']):
        print(m.group(1))

Battle of Waterloo 1815.PNG
The British Empire.png
Uk topo en.jpg
BenNevis2005.jpg
Elizabeth II greets NASA GSFC employees, May 8, 2007 edit.jpg
Palace of Westminster, London - Feb 2007.jpg
David Cameron and Barack Obama at the G20 Summit in Toronto.jpg
Soldiers Trooping the Colour, 16th June 2007.jpg
Scotland Parliament Holyrood.jpg
London.bankofengland.arp.jpg
City of London skyline from London City Hall - Oct 2008.jpg
Oil platform in the North SeaPros.jpg
Eurostar at St Pancras Jan 2008.jpg
Heathrow T5.jpg
Anglospeak.svg


## 25. テンプレートの抽出

In [7]:
match = re.search('基礎情報(.*?)^}}',eng['text'],flags=re.DOTALL+re.MULTILINE)
info = match.group(1)
templates = {}
for itr in re.finditer('^\|([^=]*)=(.*?)(?=^\|)',info,flags=re.DOTALL+re.MULTILINE):
    templates[itr.group(1).strip()]=itr.group(2).strip()
templates

{'GDP/人': '36,727<ref name="imf-statistics-gdp" />',
 'GDP値': '2兆3162億<ref name="imf-statistics-gdp" />',
 'GDP値MER': '2兆4337億<ref name="imf-statistics-gdp" />',
 'GDP値元': '1兆5478億<ref name="imf-statistics-gdp">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=70&pr.y=13&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=112&s=NGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp=0&a= IMF>Data and Statistics>World Economic Outlook Databases>By Countrise>United Kingdom]</ref>',
 'GDP統計年': '2012',
 'GDP統計年MER': '2012',
 'GDP統計年元': '2012',
 'GDP順位': '6',
 'GDP順位MER': '5',
 'ISO 3166-1': 'GB / GBR',
 'ccTLD': '[[.uk]] / [[.gb]]<ref>使用は.ukに比べ圧倒的少数。</ref>',
 '人口値': '63,181,775<ref>[http://esa.un.org/unpd/wpp/Excel-Data/population.htm United Nations Department of Economic and Social Affairs>Population Division>Data>Population>Total Population]</ref>',
 '人口大きさ': '1 E7',
 '人口密度値': '246',
 '人口統計年': '2011',
 '人口順位': '22',
 '位置画像': 'Location_UK_EU_Europe_001.svg',
 '元首等氏名': '[[エリザベス2世]]',


## 26. 強調マークアップの除去

In [8]:
for itr in re.finditer('^\|([^=]*)=(.*?)(?=^\|)',info,flags=re.DOTALL+re.MULTILINE):
    key = itr.group(1).strip()
    val = re.sub("\'\'+(.*?)\'\'+","\\1",itr.group(2).strip())
    val = re.sub("\[\[(.*?)\]\]","\\1",val)
    templates[key]=val
templates

{'GDP/人': '36,727<ref name="imf-statistics-gdp" />',
 'GDP値': '2兆3162億<ref name="imf-statistics-gdp" />',
 'GDP値MER': '2兆4337億<ref name="imf-statistics-gdp" />',
 'GDP値元': '1兆5478億<ref name="imf-statistics-gdp">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=70&pr.y=13&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=112&s=NGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp=0&a= IMF>Data and Statistics>World Economic Outlook Databases>By Countrise>United Kingdom]</ref>',
 'GDP統計年': '2012',
 'GDP統計年MER': '2012',
 'GDP統計年元': '2012',
 'GDP順位': '6',
 'GDP順位MER': '5',
 'ISO 3166-1': 'GB / GBR',
 'ccTLD': '.uk / .gb<ref>使用は.ukに比べ圧倒的少数。</ref>',
 '人口値': '63,181,775<ref>[http://esa.un.org/unpd/wpp/Excel-Data/population.htm United Nations Department of Economic and Social Affairs>Population Division>Data>Population>Total Population]</ref>',
 '人口大きさ': '1 E7',
 '人口密度値': '246',
 '人口統計年': '2011',
 '人口順位': '22',
 '位置画像': 'Location_UK_EU_Europe_001.svg',
 '元首等氏名': 'エリザベス2世',
 '元首等肩書': 'イ

## 27. 内部リンクの除去
## 28. MediaWikiマークアップの除去

In [9]:
for itr in re.finditer('^\|([^=]*)=(.*?)(?=^\|)',info,flags=re.DOTALL+re.MULTILINE):
    key = itr.group(1).strip()
    val = re.sub("\'\'+(.*?)\'\'+"," \\1 ",itr.group(2).strip())
    val = re.sub("\[\[(.*?)\]\]"," \\1 ",val)
    val = re.sub("\[(.*?)\]"," \\1 ",val)
    val = re.sub('<.*?>',"",val)
    val = re.sub('{{|}}',"",val)
    templates[key]=val
templates

{'GDP/人': '36,727',
 'GDP値': '2兆3162億',
 'GDP値MER': '2兆4337億',
 'GDP値元': '1兆5478億 http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=70&pr.y=13&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=112&s=NGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp=0&a= IMF>Data and Statistics>World Economic Outlook Databases>By Countrise>United Kingdom ',
 'GDP統計年': '2012',
 'GDP統計年MER': '2012',
 'GDP統計年元': '2012',
 'GDP順位': '6',
 'GDP順位MER': '5',
 'ISO 3166-1': 'GB / GBR',
 'ccTLD': ' .uk  /  .gb 使用は.ukに比べ圧倒的少数。',
 '人口値': '63,181,775 http://esa.un.org/unpd/wpp/Excel-Data/population.htm United Nations Department of Economic and Social Affairs>Population Division>Data>Population>Total Population ',
 '人口大きさ': '1 E7',
 '人口密度値': '246',
 '人口統計年': '2011',
 '人口順位': '22',
 '位置画像': 'Location_UK_EU_Europe_001.svg',
 '元首等氏名': ' エリザベス2世 ',
 '元首等肩書': ' イギリスの君主|女王 ',
 '公式国名': 'lang|en|United Kingdom of Great Britain and Northern Ireland英語以外での正式国名:\n*lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus 

## 29. 国旗画像のURLを取得する

参考 

- https://www.mediawiki.org/wiki/API:Imageinfo

- https://www.mediawiki.org/wiki/API:Query/ja#Getting_a_list_of_page_IDs

In [10]:
flag_file = re.sub(' ','%20',templates['国旗画像'])

query = "https://ja.wikipedia.org/w/api.php?action=query&indexpageids&titles=File:{0}&prop=imageinfo&iiprop=url&format=json".format(flag_file)
with urllib.request.urlopen(query) as res:
    line = res.read()
    res_obj = json.loads(line)
    idx = res_obj['query']['pageids'][0];
    print(res_obj['query']['pages'][idx]['imageinfo'][0]['url'])

https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg
